In [1]:
!pip3 install torch torchvision

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\danis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!python -V
!nvidia-smi

Python 3.12.7
Fri Nov  8 15:11:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.83                 Driver Version: 546.83       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8               1W /  40W |    169MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+-----------

In [3]:
import torch
print("Using torch", torch.__version__)

Using torch 2.3.1+cpu


In [4]:
!kaggle datasets download rishabkoul1/vechicle-dataset

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!unzip /content/vechicle-dataset.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
import timm
from timm.data import create_dataset
from timm.data.transforms_factory import create_transform

train_dir='/content/vechicles/train'
test_dir='/content/vechicles/test'

train_data = create_dataset(name='', root=train_dir, transform=create_transform(224, is_training=True, auto_augment='rand-m9-mstd0.5'))
test_data = create_dataset(name='', root=test_dir, transform=create_transform(224))

ModuleNotFoundError: No module named 'timm'

In [ ]:
from timm.data.loader import create_loader

In [ ]:
try:
    # only works if gpu present on machine
    train_dataloader = create_loader(train_data, input_size=224, batch_size=32, is_training=True)
    test_dataloader = create_loader(test_data, input_size=224, batch_size=32, is_training=False)
except:
    train_dataloader = create_loader(train_data, input_size=224, batch_size=32, is_training=True,use_prefetcher=False)
    test_dataloader = create_loader(test_data, input_size=224, batch_size=32, is_training=False,use_prefetcher=False)

In [ ]:
import os
class_names = sorted(entry.name for entry in os.scandir(train_dir) if entry.is_dir())
class_names

['bike',
 'boat',
 'bus',
 'car',
 'cycle',
 'helicopter',
 'plane',
 'scooty',
 'truck']

In [ ]:
m = timm.create_model('mobilenetv3_large_100', pretrained=True, exportable=True)

print(f'Original pooling: {m.global_pool}')
print(f'Original classifier: {m.get_classifier()}')

m.reset_classifier(9, 'max')

print(f'Modified pooling: {m.global_pool}')
print(f'Modified classifier: {m.get_classifier()}')

Original pooling: SelectAdaptivePool2d(pool_type=avg, flatten=Identity())
Original classifier: Linear(in_features=1280, out_features=1000, bias=True)
Modified pooling: SelectAdaptivePool2d(pool_type=max, flatten=Identity())
Modified classifier: Linear(in_features=1280, out_features=9, bias=True)


In [ ]:
import torch
from torch import nn
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)

In [ ]:
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:

    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')  # CUDA GPU
elif torch.backends.mps.is_available():
    device = torch.device('mps') #Apple GPU
else:
    device = torch.device("cpu")

In [ ]:
device

device(type='cpu')

In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results = train(model=m,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=15,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.9254 | train_acc: 0.4308 | test_loss: 1.0110 | test_acc: 0.6979


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# Plot loss curves of a model
def plot_loss_curves(results):
    loss = results["train_loss"]
    test_loss = results["test_loss"]

    accuracy = results["train_acc"]
    test_accuracy = results["test_acc"]

    epochs = range(len(results["train_loss"]))

    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

In [ ]:
plot_loss_curves(results)

In [ ]:
m1 = timm.create_model('mobilenetv3_large_100', pretrained=True, exportable=True)

print(f'Original pooling: {m1.global_pool}')
print(f'Original classifier: {m1.get_classifier()}')
print('--------------------')

m1.reset_classifier(9, 'avg')

print(f'Modified pooling: {m1.global_pool}')
print(f'Modified classifier: {m1.get_classifier()}')

In [ ]:
import torch
from torch import nn
from timm.optim import create_optimizer_v2
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = create_optimizer_v2(m1, opt='sgd', lr=0.01, momentum=0.8); optimizer, type(optimizer)

In [ ]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results2 = train(model=m1,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=15,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

In [ ]:
plot_loss_curves(results2)

In [ ]:
from typing import List, Tuple
import torchvision
from torchvision import transforms
from PIL import Image

# 1. Take in a trained model, class names, image path, image size, a transform and target device
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        class_names: List[str],
                        image_size: Tuple[int, int] = (224, 224),
                        transform: torchvision.transforms = None,
                        device: torch.device=device):


    # 2. Open image
    img = Image.open(image_path)

    # 3. Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    ### Predict on image ###

    # 4. Make sure the model is on the target device
    model.to(device)

    # 5. Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
      # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
      transformed_image = image_transform(img).unsqueeze(dim=0)

      # 7. Make a prediction on image with an extra dimension and send it to the target device
      target_image_pred = model(transformed_image.to(device))

    # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 9. Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # 10. Plot image with predicted label and probability
    plt.figure()
    plt.imshow(img)

    plt.title(f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}")
    plt.axis(False);

In [ ]:
import random
from pathlib import Path
num_images_to_plot = 3
test_dir = "/content/vechicles/test"
test_image_path_list = list(Path(test_dir).glob("*/*.jpg")) # get list all image paths from test data

In [ ]:
test_image_path_sample = random.sample(population=test_image_path_list, # go through all of the test image paths
                                       k=num_images_to_plot) # randomly select 'k' image paths to pred and plot

In [ ]:
test_image_path_sample

In [ ]:
# Make predictions on and plot the images
for image_path in test_image_path_sample:
    pred_and_plot_image(model=m1,
                        image_path=image_path,
                        class_names=class_names,
                        # transform=weights.transforms(), # optionally pass in a specified transform from our pretrained model weights
                        image_size=(224, 224))

In [ ]:
!pip install onnx

In [ ]:
!pip install onnxruntime-gpu onnx onnxconverter_common==1.8.1 pillow

In [ ]:
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime
import onnx
from onnx import numpy_helper
import urllib.request
import json
import time

# display images in notebook
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

%matplotlib inline

In [ ]:
img, label = test_data[0]

In [ ]:
img.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.MFD_HUGE_1GB = m1.to(device)

In [ ]:
x = torch.randn(2, 3, 224, 224, requires_grad=True).to(device)

In [ ]:
#x = torch.randn(2, 3, 224, 224, requires_grad=True)
torch_out = m1(x)

# Export the model
torch.onnx.export(m1,                                       # model being run
                  x,                                           # model input (or a tuple for multiple inputs)
                  "imagenet_large100_vehicle.onnx",                            # where to save the model (can be a file or file-like object)
                  export_params=True,                          # store the trained parameter weights inside the model file
                  opset_version=10,                            # the ONNX version to export the model to
                  do_constant_folding=True,                    # whether to execute constant folding for optimization
                  input_names = ['input'],                     # the model's input names
                  output_names = ['output'],                   # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},  # variable length axes
                                'output': {0 : 'batch_size'}})

In [ ]:
import onnx

onnx_model = onnx.load("imagenet_large100_vehicle.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
onnx_model = onnx.load("imagenet_large100_vehicle.onnx")

In [ ]:
onnx.checker.check_model(onnx_model)

In [ ]:
traced_model=torch.jit.trace(m1, torch.rand(8,3,224,224).to(device))

In [ ]:
traced_output=traced_model(torch.rand(8,3,224,224).to(device))
traced_output.shape

In [ ]:
# Run the model on the backend
session = onnxruntime.InferenceSession('imagenet_large100_vehicle.onnx', None)

# get the name of the first input of the model
input_name = session.get_inputs()[0].name

print('Input Name:', input_name)

In [ ]:
def load_labels(path):
    with open(path) as f:
        data = json.load(f)
    return np.asarray(data)

def preprocess(input_data):
    # convert the input data into the float32 input
    img_data = input_data.astype('float32')

    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]

    #add batch channel
    norm_img_data = norm_img_data.reshape(1, 3, 224, 224).astype('float32')
    return norm_img_data

def softmax(x):
    x = x.reshape(-1)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def postprocess(result):
    return softmax(np.array(result)).tolist()

In [ ]:
filename = '/content/vechicles/test/bus/Z (11).jpg'
input_image = Image.open(filename)

In [ ]:
input_image

In [ ]:
newsize = (224, 224)
input_image = input_image.resize(newsize)
# Shows the image in image viewer
input_image.show()

In [ ]:
input_image

In [ ]:
m1 = m1.to(device)
x = torch.randn(8, 3, 224, 224, requires_grad=True).to(device)

traced_model = torch.jit.trace(m1, x)

In [ ]:
labels = class_names

print("Image size: ", input_image.size)
plt.axis('off')
display_image = plt.imshow(input_image)
image_data = np.array(input_image).transpose(2, 0, 1)
image_data.shape

In [ ]:
input_data = preprocess(image_data)

In [ ]:
input_data.shape

In [ ]:
start = time.time()
raw_result = session.run([], {input_name: input_data})
end = time.time()
res = postprocess(raw_result)

In [ ]:
inference_time = np.round((end - start) * 1000, 2)
idx = np.argmax(res)

print('========================================')
print('Final top prediction is: ' + labels[idx])
print('========================================')

print('========================================')
print('Inference time: ' + str(inference_time) + " ms")
print('========================================')

sort_idx = np.flip(np.squeeze(np.argsort(res)))
print('============ Top 5 labels are: ============================')
print([labels[i] for i in sort_idx[:5].tolist()])
print('===========================================================')

plt.axis('off')
display_image = plt.imshow(input_image)

In [ ]:
type(traced_model)

In [ ]:
traced_model(torch.rand(8, 3, 224, 224).to(device)).shape

In [ ]:
filename = '/content/vechicles/test/bus/Z (11).jpg' # change to your filename
from torchvision import models, datasets, transforms as T
import torch
from PIL import Image
import numpy as np

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)
print("GPU Availability: ", torch.cuda.is_available())
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')

In [ ]:
# Inference with ONNX Runtime
import onnxruntime
from onnx import numpy_helper
import time

session_fp32 = onnxruntime.InferenceSession("imagenet_large100_vehicle.onnx", providers=['CPUExecutionProvider'])

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def run_sample(session, categories, inputs):
    latency = []
    start = time.time()
    input_arr = inputs.cpu().detach().numpy()
    ort_outputs = session.run([], {'input':input_arr})[0]
    latency.append(time.time() - start)
    output = ort_outputs.flatten()
    output = softmax(output) # this is optional
    top5_catid = np.argsort(-output)[:5]
    for catid in top5_catid:
        print(categories[catid], output[catid])
    return ort_outputs, latency

ort_output, latency = run_sample(session_fp32, class_names, input_batch)
print("ONNX Runtime CPU/GPU/OpenVINO Inference time = {} ms".format(format(sum(latency) * 1000 / len(latency), '.2f')))